In [1]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import shapely.ops
import rasterio
import rasterio.mask
import numpy
import matplotlib.pyplot
import scipy.ndimage
import scipy.interpolate
import pathlib
import pdal
import json

# Setup paths

In [2]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\Small_test_site')
combined_path = pathlib.Path(r'combined_data')

lidar_name = pathlib.Path(r'points.laz')
dem_name = pathlib.Path(r'combined_dem.nc')

# Read in LiDAR with relevant processing
Load in the LAZ file with relevant processing
*  Set projection: https://pdal.io/stages/filters.reprojection.html#filters-reprojection

In [7]:
crs = 2193
lidar_file_name = base_path/combined_path/lidar_name

resolution = 20
dem_file_name = base_path/combined_path/dem_name

pdal_pipeline_instructions = [
    {"type":  "readers.las", "filename": str(lidar_file_name)},
    {"type":"filters.reprojection","out_srs":"EPSG:" + str(crs)}, # reproject to NZTM
    {"type":  "writers.gdal", "filename": str(dem_file_name), "resolution": resolution, "window_size": 5,"output_type":"mean",}
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

In [16]:
metadata=json.loads(pdal_pipeline.get_metadata())
lidar_arrays = pdal_pipeline.arrays

# Look at creating a DEM
There a re a few options here. 
* Look for routine in a library like PDAL
* Code up a simple routine myself using a KDTree or something similar

`tree = scipy.spatial.KDTree(numpy.c_[pdal_pipeline.arrays[0]['X'].ravel(), pdal_pipeline.arrays[0]['X'].ravel()])`

In [18]:
resolution = 10
dem_file_name = base_path/combined_path/dem_name

pdal_pipeline_instructions = [
    {"type":  "writers.gdal", "filename": str(dem_file_name), "resolution": resolution, "gdalopts":"a_srs=EPSG:" + str(crs)}#,"gdaldriver": "netCDF"}
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), lidar_arrays)
pdal_pipeline.execute();